In [ ]:
#extract filenames from /content/xml/epidoc and write them into a csv file
# NB! replace directory paths as appropriate

import glob
import csv

mylist = [f for f in glob.glob("/Users/simonastoyanova/Desktop/ircyr-efes/webapps/ROOT/content/xml/epidoc/*.xml")]
mylist.sort()
#print(mylist)


with open("/Users/simonastoyanova/Desktop/ircyr-test/epidoc-dir.csv", 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in mylist:
        wr.writerow([word])


In [ ]:
#check filename and content of <idno type="filename"> for consistency

import re

with open("epidoc-dir.csv", 'r') as listfiles:
    lf = listfiles.readlines()
   # print(lf)
    for x in lf:
        x = x.rstrip()
        if len(x.split('.')[0]) != 6:
            print("ERROR in " + x + " malformed filename!")
        if x !="Filenames":
            #print(x)
            #fpath = '/Users/simonastoyanova/Desktop/ircyr-test/input/' + x     #ADD EXCEPTION HERE
            fpath = '/Users/simonastoyanova/Desktop/ircyr-efes/webapps/ROOT/content/xml/epidoc/' + x     #ADD EXCEPTION HERE
            infile = open(fpath,"r")
            contents = infile.read()
            idnos = re.findall(r'<idno type="filename">(.*?)</idno>', contents)
            if len(idnos) > 1:
                print("ERROR in " + x + " too many idnos!")
            elif len(idnos) == 0:
                print("ERROR in " + x + " no idno!")
            if len(idnos[0]) != 6:
                print("ERROR in " + x + " malformed idno ("+idnos[0]+") !")
            if x.split('.')[0] != idnos[0]:
                print("ERROR in " + x + " idno ("+idnos[0]+") doesn't match filename!")

                